In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader
from PIL import Image
import requests
from io import BytesIO

In [2]:
# Define the transformations for the CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # VGG expects 224x224 input size
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [3]:
# Load CIFAR-10 dataset
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=4)


100%|██████████| 170M/170M [00:18<00:00, 9.09MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [4]:
# Define the VGGNet model (using VGG16 as an example)
class VGGNet(nn.Module):
    def __init__(self):
        super(VGGNet, self).__init__()
        self.vgg = models.vgg16(pretrained=True)  # Load pre-trained VGG16
        self.vgg.classifier[6] = nn.Linear(4096, 10)  # Change the final layer to match CIFAR-10 classes

    def forward(self, x):
        return self.vgg(x)

In [5]:
# Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VGGNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:09<00:00, 59.1MB/s]


In [6]:
# Training the model
for epoch in range(5):  # Train for a number of epochs
    print(f'Training epoch: {epoch + 1}...')
    running_loss = 0.0

    model.train()  # Set the model to training mode
    for i, data in enumerate(train_dataloader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Zero the parameter gradients

        outputs = model(inputs)  # Forward pass
        if outputs is None:  # Debugging check
            print("Model output is None")
            continue

        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize the model parameters

        running_loss += loss.item()

    print(f'Loss: {running_loss / len(train_dataloader):.4f}')


Training epoch: 1...
Loss: 0.4657
Training epoch: 2...
Loss: 0.2234
Training epoch: 3...
Loss: 0.1444
Training epoch: 4...
Loss: 0.0967
Training epoch: 5...
Loss: 0.0643


In [7]:
# Evaluate the model
correct = 0
total = 0

model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        if outputs is None:  # Debugging check
            print("Model output is None during evaluation")
            continue

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = (correct / total) * 100
print(f'Test Accuracy: {accuracy:.2f}%')


Test Accuracy: 93.22%


In [8]:
class_names = ['plane','car','bird','cat','deer','dog','frog','horse','ship','truck']

In [9]:
new_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 32x32
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize
])

# Function to load and transform the image
def load_img(img_path):
    response = requests.get(img_path)
    image = Image.open(BytesIO(response.content))  # Open image from byte data
    image = new_transform(image)  # Apply transformations
    image = image.unsqueeze(0)  # Add batch dimension
    return image

# Image URLs
image_paths = [
    'https://www.nylabone.com/-/media/project/oneweb/nylabone/images/dog101/10-intelligent-dog-breeds/golden-retriever-tongue-out.jpg',
    'https://media.gq.com/photos/6508829d305ef4e0229049b3/master/w_2240,c_limit/plane.jpg',
    'https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcQjr-6m4fJFTmH4sIqOHIMJbQBTMw4JhG-CcVscjg5N-fDXF6ku',
    'https://cdn.speedsize.com/eb8d0010-7300-4129-8a6d-74bc221f9caf/https://www.virginvoyages.com/dam/jcr:820ffd9b-f003-4e99-84f1-cc3e592b9da3/scarlet%20lady-2252x1266.png',
    'https://woofwell.com/cdn/shop/files/Golden-Retriever-Health-WoofWell-Breed-Specific-Dog-Supplements_1600x.jpg?v=1621360789',
    'https://i.redd.it/f0mauu1qpbz41.jpg',
    'https://www.nylabone.com/-/media/project/oneweb/nylabone/images/dog101/top-10-lists/10-intelligent-dog-breeds.jpg?h=318&iar=0&w=720&hash=BDE1A53E84C77A8C1C4DA40F79DE0915'
]

# Load images
images = [load_img(img) for img in image_paths]

# Model evaluation
model.eval()
with torch.no_grad():
    for image in images:
        image = image.to(device)
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
        print(f'The prediction: {class_names[predicted.item()]}')

The prediction: dog
The prediction: plane
The prediction: cat
The prediction: ship
The prediction: dog
The prediction: ship
The prediction: dog
